# 基本的な使い方

In [1]:
# state_machine.pyからStateMachineをimport
from state_machine import StateMachine

# 遷移を記述したcsvを用意 
transition_csv = r"./transition_demo.csv"
"""
遷移を記述したcsvの作り方。
Step1:状態遷移図を作る。
Step2:状態遷移図を元に状態遷移を記したcsvを作成する。
ヘッダーを「Source,Destination,Trigger」としたcsvを用意。
Sourceは遷移元、Destinationは遷移先、Triggerはこの遷移のトリガー名(実行するためのキーワード)
状態遷移図の遷移(矢印)をせっせとデータ化する。
"""

# 状態遷移を司るステートマシンを作成。
# transitionライブラリで作るステートマシンは非常に多機能で、同じ動作でも複数のコーディング方法があったり
# transitionの仕様の把握が必要な暗黙的な処理が多いので、作り込むと複雑すぎてメンテも大変だし可読性も下がる。
# このライブラリはシンプルに状態遷移の機能と作成したステートマシンの図示や挙動確認だけを提供する。
stmach = StateMachine(transition_csv=transition_csv, initial_state="BOOT",
                      use_graph=True, show_transition=True)
""" StateMachineの引数は以下
transition_csv:先述の遷移を記述したcsv
initial_state:状態遷移の初期状態(string型)
use_graph:図示系の機能を使用するかどうか。使うと動作が遅くなるらしいので、デバッグ用に。
          動作にはgraphvizのセットアップが必要。詳細は元リポジトリで
          https://github.com/pytransitions/transitions
show_transition:実行した遷移をprint表示するかどうか
"""
# ステートマシンで定義した状態遷移を図示
stmach.view_state_transition()

# 状態の遷移はステートマシンのtriggerで実行する。
# 引数に実行したい遷移のトリガー名を入れる。
stmach.trigger("TO_IDLING")  # TO_IDLINGトリガーで定義された状態遷移を実行
stmach.trigger("TO_INSPECT")  # TO_INSPECTトリガーで定義された状態遷移を実行
stmach.trigger("EXCEPTION")  # EXCEPTIONトリガーで定義された状態遷移を実行
stmach.trigger("RESET")  # RESETトリガーで定義された状態遷移を実行
# 上記はBOOT => IDLING => INSPECTION => ERROR => BOOTの遷移になる。

# おわり

TO_IDLING : BOOT => IDLING
TO_INSPECT : IDLING => INSPECTION
EXCEPTION : INSPECTION => ERROR
RESET : ERROR => BOOT


True

# ステートマシンの状態をグラフィカルに観察したい！

In [ ]:
from state_machine import StateMachine
import time

transition_csv = r"./transition_demo.csv"
stmach = StateMachine(transition_csv, initial_state="BOOT", use_graph=True, show_transition=False)

# BOOT => IDLING => INSPECTION => ERROR => BOOTの遷移を永久に回る。
trigger_set = ["TO_IDLING", "TO_INSPECT", "EXCEPTION", "RESET"]
while(1):
    for trig in trigger_set:
        stmach.trigger(trig)
        stmach.save_state_transition_pngimg("state_transition")
        time.sleep(1)
        
"""
出力した状態遷移図は現状態や遷移も図示される。
VSCODEでstate_transition.pngを開くと、1秒ごとに遷移が進んでいくので楽しい。
動画として保存したいときは、タイムスタンプを画像名として遷移毎に保存していって
cv2でも使って動画にすればいいと思います。
"""


# 定義外のトリガーの挙動

In [ ]:
from state_machine import StateMachine
import time

"""
遷移csvに記述されていない定義外の遷移トリガを実行すると、例外が発生してしまいます。
誤命令で例外としてソフトウェアが終了するのが不都合な場合がありますので
その場合は定義外のトリガーかどうかを事前に確認する処理が有効です。
"""

# ステートマシンを定義
transition_csv = r"./transition_demo.csv"
stmach = StateMachine(transition_csv, initial_state="BOOT", use_graph=True, show_transition=True)

# 定義外のトリガーの例として
# MAINTENANCE状態には遷移しないINSPECTION状態にTO_MAINTEトリガを入れる。
trigger_set = ["TO_IDLING", "TO_INSPECT", "TO_MAINTE", "EXCEPTION", "RESET"]
for trig in trigger_set:
    if stmach.is_valid_transition(trig): #現状態とトリガーの組み合わせが正しいかどうか
        stmach.trigger(trig)
    else:
        print("状態%sにトリガ%sによる遷移は定義されていません。"%(stmach.state,trig))
        """
        実際の用途として、ここにエラー状態への遷移などを追記してもいいと思います。
        transitionライブラリの仕様上,定義した遷移先を同トリガ内で、別の遷移先に変更する実装は難しいです。
        遷移した後に、更に別状態に遷移するようなコールバックを仕込むことはできますが
        最初に指定した遷移後に別状態に遷移することになるので、個人的には好きではないです。
        設計の状態遷移図と挙動が違うことになるので。
        また、コールバックの優先順位やらガード判定やらややこしくなるので。これらの特別な処理を標準とはしません。
        """
